En este notebook usaremos llama 3 a través de una API (uso "limitado" gratuito), sin usar el llm localmente en la computadora. Es necesario crear una cuenta aquí:

https://console.groq.com/login

y solicitar una key:

https://console.groq.com/keys

Create a file named .env where you store the value of your key:

GROQ_API_KEY='---'

In [1]:
!pip install groq

In [2]:
!pip install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


In [9]:
!pip install langchain_groq

  Using cached langchain_core-0.3.28-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.28-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21


In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [5]:
from dotenv import load_dotenv
load_dotenv()
api_key= os.getenv('GROQ_API_KEY')

In [6]:
model_name = 'llama-3.1-70b-versatile'

groq_chat = ChatGroq(
        groq_api_key=api_key,
        model_name=model_name
    )

llm = groq_chat

In [7]:
## Do not mofify
def load_db(embeddings,pdf_path):
    text =''
    with open(pdf_path,'rb') as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter=SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile")
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_text(text)
    # define embedding
    vectorstore = FAISS.from_texts(docs, embeddings)
    return vectorstore

In [8]:
embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')

load INSTRUCTOR_Transformer
max_seq_length  512


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\.venv\Lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

In [9]:
pdf_path = "PeterPan.pdf"

In [10]:
#Do not modify 
import os
if not os.path.exists('faiss_index'):
    vectorstore=load_db(embeddings,pdf_path)
    vectorstore.save_local("faiss_index")
else:
    vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)


In [11]:
retriever = vectorstore.as_retriever()

In [12]:
template = """Eres un asistente para contestar preguntas del ususario sobre el contenido del file subido, que trata de la historia de Peter Pan. 
    Contesta siempre en español y agradece el usuario por preguntar. Si las preguntas son sobre otro tema, contesta que no puedes contestar.
    {context}
    Question: {question}
    Helpful Answer:"""
qa_prompt = ChatPromptTemplate.from_template(template)

In [13]:
#Do not modify
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

C:\Users\L01257675\AppData\Local\Temp\ipykernel_7864\3110315447.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [14]:
history = []

In [15]:
def handle_conversation(conversation,history):
    print('Escribe "exit" para salir')
    while True:
        question = input('Tu: ')
        if question.lower() == 'exit':
            break
        print(question)
        result = conversation({"question": question}, {"chat_history": history})
        history.append((question, result["answer"]))
        print("Bot: ",result['answer'])

handle_conversation(conversation_chain,history)

Escribe "exit" para salir
¿De qué trata la historia? Haz un resumen en español, por favor.


C:\Users\L01257675\AppData\Local\Temp\ipykernel_7864\387802828.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation({"question": question}, {"chat_history": history})


Bot:  La historia trata sobre las aventuras de Peter Pan, un niño que nunca crece, y su relación con la familia Darling, especialmente con Wendy y sus hermanos Michael y John. La historia comienza cuando Peter Pan llega a la casa de los Darling en busca de su sombra, que había quedado atrapada allí. Con la ayuda de Wendy, Peter recupera su sombra y se gana la amistad de los niños.

Posteriormente, Peter les invita a unirse a él en el País de Nunca Jamás, un lugar mágico donde los niños nunca crecen y viven aventuras emocionantes. Allí, se encuentran con los Niños Perdidos, un grupo de niños que viven con Peter, y también con piratas, hadas y otros seres mágicos.

Sin embargo, la felicidad no dura mucho, ya que el capitán Garfio, un pirata que busca venganza contra Peter, intenta capturar a los niños y hacerles daño. Peter y sus amigos deben enfrentar a Garfio y sus piratas en una serie de aventuras emocionantes y peligrosas.

En resumen, la historia es una aventura emocionante y llena 